## Rider Sentiment Analysis


# After webscraping the data, we will perform sentiment analysis on the reviews of the riders.
#### The data is collected from various sources including:
- https://www.reddit.com/r/transit/comments/1bt9yng/the_chicago_l_is_an_american_treasure/
- https://www.reddit.com/r/cta/comments/1dwa1w5/how_bad_is_the_cta_really_compared_to_other_cities/
- https://www.tripadvisor.com/ShowUserReviews-g35805-d549354-r491931481-Chicago_Transit_Authority_CTA-Chicago_Illinois.html
- https://www.yelp.com/biz/chicago-transit-authority-chicago-6
#### Employee reviews from:
https://www.indeed.com/cmp/Chicago-Transit-Authority/reviews




In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json

import requests
from bs4 import BeautifulSoup

# Webscraping the data from reddit

def get_reddit_comments(url):
    """
    Function to get reddit comments from a given url
    """
    # Send a GET request to the URL
    response = requests.get(url, headers={'User-Agent': 'Chrome/58.0.3029.110'})
    # Find all the comments in the post
    data = response.json()
    
    comments_body = [
        comment['data']['body']
        for comment in data[1]['data']['children']
        if 'body' in comment['data']
    ]
    return comments_body

# Define the URLS
reddit_urls = [
    'https://www.reddit.com/r/transit/comments/1bt9yng/the_chicago_l_is_an_american_treasure/.json',
    'https://www.reddit.com/r/cta/comments/1dwa1w5/how_bad_is_the_cta_really_compared_to_other_cities/.json',
    'https://www.reddit.com/r/cta/comments/1gjox52/what_is_in_your_opinion_the_worst_designed_cta_l/.json',
    
]

# join get_reddit_comments(rdt_url) and get_reddit_comments(rdt_url2)
comments_body = []
for url in reddit_urls:
    comments_body.extend(get_reddit_comments(url))

for comment in comments_body:
    print(comment)
    print("\n")
    
print(len(comments_body))
# Store the post & comments in a list


# Do the same for the other sources


I love it. It's like a museum artifact that doubles as a transit system. Parts of it for sure need upgrades, and the city could benefit from some new, modern metro-style lines like Sydney and Toronto are building, but the character of the current L is such an iconic part of the city's character.

Question though - to what extent is our affinity for the L limited to the actual elevated parts? Like, are the subways under State and Dearborn considered as 'special' as the turn-of-the-century style elevated portions of the network?


It needs to be upgraded, with an economy bigger then most countries on earth Chicago should be able to make every station look modern and better


I had a higher opinion of the L before I had to ride it.


Perhaps it could be intriguing to preserve one station as a 'Historical Gem' to honor its original design and significance. However, for the hundreds of thousands of users, as well as those working, walking, or living nearby, the overall state of disrepair th